In [ ]:
!touch pytorch/train.py
!touch pytorch/model.py
!touch pytorch/predict.py
!touch pytorch/requirements.txt

In [ ]:
import pandas as pd
pd.read_csv(local_data_folder + 's3/cluster-0/train.csv', header=None).shape

In [ ]:
# Build model using PyTorch
import sagemaker
from sagemaker.pytorch import PyTorch

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
ticker = 'A'
local_data_folder = './data/'
prefix = "udacity-capstone-project"

#cluster-0
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/data/cluster-0/train.csv'.format(bucket, prefix), content_type='text/csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/data/cluster-0/validation.csv'.format(bucket, prefix), content_type='text/csv')
                    
estimator = PyTorch(entry_point='train.py',
                    source_dir='pytorch', # this should be just "source" for your code
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_dim': 26,  # num of features
                        'hidden_dim': 260,
                        'output_dim': 1,
                        'epochs': 200 # could change to higher
                    })

estimator.fit({ 'train': s3_input_train, 'validation': s3_input_validation })